# Capstone project

1. Discussion and introduction to the business problem;
2. Target audience;
3. Data analysis;
4. Results and conclusion;




## 1. Discussion and introduction to the business problem

London is one of the financial cities in the world, with its gross regional product of £500 bil. it forms a quarter of the UKs GDP. It also happens to be one of the most populated and diverse with it's 9.3 mil and 37% foreign born. It is a perfect place to start your business. The finance industry is located in the following districts: the City, Westminster, Canary Wharf, Camden & Islington and Lambeth & Southwark. These are the areas we will be focusing on. The idea is to open a place that serves lunch to the nearby office employees. Due to the high density of office buliding the flow of customers would be colossal.

Further in the project we will look at the current available restaurants in the areas, it's speciality and condensity using the Forsquare application, projecting it on to the map for a more convenient view. Based on the analysis, we will detemine the indicators that will allow to make a decission on the future joint.

## 2. Target audience

1. Investors looking to open a new restaurant;
2. Office workers looking for a place to eat in;
3. Restaurant owners in the areas to check for competitors;


